In [26]:
def approx_match(time_a, time_b, slippage=0.040000000000000036):
    return abs(time_a - time_b) <= slippage

In [28]:
SAMPLE = "2442204240010034621"

In [34]:
from pathlib import Path
TEXT = Path("/Users/joregan/Playing/kbw2v")
PHONES = Path("/Users/joregan/Playing/rd_phonetic")

In [35]:
text = open(TEXT / f"{SAMPLE}_480p.json")
phones = open(PHONES / f"{SAMPLE}_480p.json")

In [36]:
import json
text_json = json.load(text)
phone_json = json.load(phones)

In [50]:
class Chunk:
    def __init__(self, chunk):
        self.text = chunk['text']
        self.start = chunk['timestamp'][0]
        self.end = chunk['timestamp'][1]
    
    def __repr__(self) -> str:
        return f"[{self.text}: {self.start}, {self.end}]"

class SimpleMerge(Chunk):
    def __init__(self, left: Chunk, right: Chunk):
        self.text = left.text
        self.phone = right.text
        self.start = left.start
        self.end = left.end
        self.diff_start = left.start - right.start
        self.diff_end = left.end - right.end

    def exact_length(self):
        return self.diff_start == 0 and self.diff_end == 0
        

In [38]:
text_chunks = []
phone_chunks = []
for text_chunk in text_json['chunks']:
    text_chunks.append(Chunk(text_chunk))

for phone_chunk in phone_json['chunks']:
    phone_chunks.append(Chunk(phone_chunk))

In [51]:
def approx_match_chunks(left, right):
    return approx_match(left.start, right.start) and approx_match(left.end, right.end)

In [55]:
def print_pair(left, right):
    print(f"{left.text} :: {right.text} ({left.start}, {left.end}; {right.start}, {right.end})")

In [56]:
tci = 0
pci = 0

merged = []

while tci < len(text_chunks) and pci < len(phone_chunks):
    if approx_match_chunks(text_chunks[tci], phone_chunks[pci]):
        merged.append(SimpleMerge(text_chunks[tci], phone_chunks[pci]))
    elif approx_match(text_chunks[tci].start, phone_chunks[pci].start):
        save_pci = pci
        while not approx_match(text_chunks[tci].end, phone_chunks[pci].end):
            pci += 1
    else:
        print_pair(text_chunks[tci], phone_chunks[pci])
    tci += 1
    pci += 1

IX :: iː (0.02, 0.34; 0.02, 0.06)
DEBATT :: seks (0.66, 1.08; 0.12, 0.46)
MED :: debat (1.18, 1.24; 0.66, 1.12)
ANLEDNING :: meː (1.38, 1.92; 1.16, 1.22)
AV :: anleːdnɪŋ (2.04, 2.18; 1.36, 1.92)
INTEPELATIONSSVAR :: ɑːv (2.38, 3.34; 2.04, 2.22)
OCH :: ɪntepelaɧuːnsvɑːr (3.84, 3.92; 2.38, 3.34)
KAMMAREN :: oː (3.96, 4.32; 3.86, 3.88)
ÖVERGÅR :: kamarən (4.44, 4.78; 3.96, 4.34)
NU :: øːvərɡoːr (4.86, 4.9; 4.42, 4.8)
TILL :: nʉː (4.98, 5.12; 4.86, 4.9)
ATT :: tɪl (5.16, 5.26; 5.04, 5.12)
DEBATTERA :: at (5.32, 5.78; 5.14, 5.24)
STATSRÅDENS :: debateːra (5.88, 6.5; 5.32, 5.8)
SVAR :: stasroːdən (6.56, 6.84; 5.86, 6.44)
PÅ :: svɑːr (6.94, 7.0; 6.54, 6.86)
MÖTENAS :: poː (7.1, 7.62; 6.96, 7.02)
INTERPELATIONER :: møːtæɳa (8.34, 9.16; 7.08, 7.48)
OCH :: s (9.66, 9.72; 7.58, 7.62)
VI :: ɪntepelaɧuːnər (9.76, 9.8; 8.34, 9.18)
STARTAR :: <pa> (9.88, 10.2; 9.48, 9.5)
MED :: oː (10.24, 10.3; 9.68, 9.7)
ARBETSMARKNADSOCH :: viː (10.42, 11.74; 9.76, 9.8)
JÄMSTÄLLDHETSMINISTER :: stɑːʈa (12.0, 13.2; 

In [54]:
merged

[]